In [1]:
!pip install python-telegram-bot --upgrade

     |████████████████████████████████| 358kB 709kB/s eta 0:00:01


In [0]:
#colab не может работать с версиями новее
!pip uninstall tornado
!pip install tornado==4.5.3

Uninstalling tornado-6.0.3:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/tornado-6.0.3.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tornado/*
Proceed (y/n)? y
  Successfully uninstalled tornado-6.0.3
     |████████████████████████████████| 491kB 3.4MB/s 
  Created wheel for tornado: filename=tornado-4.5.3-cp36-cp36m-linux_x86_64.whl size=433070 sha256=51037c785d4516a69ed504615507691e2acc7a56728eb4d2621f938a38c95a77
  Stored in directory: /root/.cache/pip/wheels/72/bf/f4/b68fa69596986881b397b18ff2b9af5f8181233aadcc9f76fd
Successfully built tornado
ERROR: python-telegram-bot 12.3.0 has requirement tornado>=5.1, but you'll have tornado 4.5.3 which is incompatible.


In [2]:
import logging
import telegram
from telegram.error import NetworkError, Unauthorized
from time import sleep

from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

import torch
import numpy as np
import PIL
from PIL import Image

In [5]:
import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

from matplotlib import colors, pyplot as plt
%matplotlib inline

In [8]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cpu")

In [12]:
#model init
model_ft = torch.load('../models/m81.pth', map_location={'cuda:0': 'cpu'})
model_ft.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
def predict_one_sample(model, inputs, device=DEVICE):
    """Предсказание, для одной картинки"""
    with torch.no_grad():
        inputs = inputs.to(device)
        model.eval()
        logit = model(inputs).cpu()
        print(logit.shape)
        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()
    return probs

In [14]:
def get_weather(model, image):
  transform = transforms.Compose([
              transforms.Resize((RESCALE_SIZE, RESCALE_SIZE)),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
          ])
  image = transform(image)
  pred = predict_one_sample(model, image.unsqueeze(0))
  label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))
  preds = label_encoder.inverse_transform(np.argmax(pred, axis=1))
  return preds

In [15]:
def load_sample(file):
        image = Image.open(file).convert('RGB')
        image.load()
        return image

In [0]:
def get_help(bot, update):
    update.message.reply_text("Send photo to get clothes recommendation")

def image_handler(bot, update):
    global model_ft
    print(update.message)
    img_path = "image1.jpg"
    img_id = update.message.photo[-1].file_id
    print(img_id)
    img = bot.get_file(img_id)
    img.download(img_path)
    img = load_sample(img_path)
    weather = get_weather(model_ft, img)
    print(weather)
    update.message.reply_text(weather[0])

def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("1056944645:AAELDA_hclG4RV402WNw89IE9TLt25F_OIM")

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # simple start function
    dp.add_handler(MessageHandler(Filters.photo, image_handler))
    dp.add_handler(CommandHandler("help", get_help))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

if __name__ == '__main__':
    main()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details


{'message_id': 30, 'date': 1581001036, 'chat': {'id': 846020196, 'type': 'private', 'username': 'dmitriy_vahrushev', 'first_name': 'Dmitriy', 'last_name': 'Vahrushev'}, 'entities': [], 'caption_entities': [], 'photo': [{'file_id': 'AgACAgIAAxkBAAMeXjwpTP91f1rT8KKcgD6tgn9iCtcAAuKtMRswvuFJlfJ8It2a64kKd8EPAAQBAAMCAANtAANxHgQAARgE', 'width': 240, 'height': 320, 'file_size': 16349}, {'file_id': 'AgACAgIAAxkBAAMeXjwpTP91f1rT8KKcgD6tgn9iCtcAAuKtMRswvuFJlfJ8It2a64kKd8EPAAQBAAMCAAN4AANyHgQAARgE', 'width': 599, 'height': 800, 'file_size': 81730}, {'file_id': 'AgACAgIAAxkBAAMeXjwpTP91f1rT8KKcgD6tgn9iCtcAAuKtMRswvuFJlfJ8It2a64kKd8EPAAQBAAMCAAN5AANzHgQAARgE', 'width': 956, 'height': 1276, 'file_size': 189971}], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': False, 'channel_chat_created': False, 'from': {'id': 846020196, 'first_name': 'Dmitriy', 'is_bot': False, 'last_name': 'Vahrushev', 'username': 'dmitriy_vahrushev